In [1]:
import pandas as pd
import dask.dataframe as dd

In [10]:
df = dd.read_parquet('..\\full_data\\figma_plan\\full_gender_data.parquet').compute()

display(df.head())
display(df.info())

,user_id,region_name_0,region_name_1,region_name_2,region_name_3,region_name_4,region_name_5,region_name_6,region_count,city_name_0,...,megapteka-ru.turbopages.org_frac,127.0.0.1_frac,bukvaprava.ru_frac,belnovosti-by.turbopages.org_frac,ngs24-ru.turbopages.org_frac,forum.mfd.ru_frac,yomed.ru_frac,7info.ru_frac,other_female_urls_frac,other_male_urls_frac
0,0,0,0,0,0,0,0,1,1,0,...,0.000105,0.000177,0.000573,0.000077,0.000077,0.016760,0.001179,0.000386,0.005181,0.000000
1,1,0,0,0,0,0,0,1,3,0,...,0.000187,0.000077,0.000112,0.009816,0.000740,0.000375,0.000225,0.001665,0.001910,0.000955
2,2,0,0,0,0,0,1,0,1,0,...,0.000143,0.000114,0.000674,0.000465,0.001210,0.000553,0.000247,0.003367,0.000000,0.000000
3,3,0,0,0,0,0,1,1,1,0,...,0.000780,0.054767,0.000249,0.000058,0.001447,0.000130,0.000198,0.000069,0.003636,0.003636
4,4,0,0,0,0,1,0,0,5,0,...,0.000172,0.000138,0.001732,0.001241,0.000484,0.027652,0.000965,0.000112,0.003861,0.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Columns: 3036 entries, user_id to other_male_urls_frac
dtypes: category(26), float32(3007), int32(1), int8(2)
memory usage: 4.7 GB


None

In [3]:
nan_urls_frac = df.isna().sum()[df.isna().sum() > 0].drop('price')
display(nan_urls_frac.head())

googleads.g.doubleclick.net_frac    20755
yandex.ru_frac                      28912
i.ytimg.com_frac                    34049
vk.com_frac                         41219
avatars.mds.yandex.net_frac         32625
dtype: int64

In [4]:
df[nan_urls_frac.index]

,googleads.g.doubleclick.net_frac,yandex.ru_frac,i.ytimg.com_frac,vk.com_frac,avatars.mds.yandex.net_frac,ad.mail.ru_frac,yastatic.net_frac,apple.com_frac,instagram.com_frac,ads.adfox.ru_frac,...,realblog-24.ru_frac,psy-magic.org_frac,megapteka-ru.turbopages.org_frac,127.0.0.1_frac,bukvaprava.ru_frac,belnovosti-by.turbopages.org_frac,ngs24-ru.turbopages.org_frac,forum.mfd.ru_frac,yomed.ru_frac,7info.ru_frac
0,0.196891,0.031088,0.025907,0.041451,0.046632,0.031088,0.051813,0.117714,0.003861,0.025907,...,0.000847,0.001109,0.000105,0.000177,0.000573,0.000077,0.000077,0.016760,0.001179,0.000386
1,0.072588,0.112703,0.040115,0.023878,0.104107,0.034384,0.081184,0.096916,0.100249,0.019102,...,0.001476,0.000525,0.000187,0.000077,0.000112,0.009816,0.000740,0.000375,0.000225,0.001665
2,0.092457,0.046229,0.126521,0.141119,0.072993,0.038929,0.046229,0.093190,0.001225,0.026764,...,0.001630,0.000566,0.000143,0.000114,0.000674,0.000465,0.001210,0.000553,0.000247,0.003367
3,0.021818,0.061818,0.007273,0.178182,0.094545,0.080000,0.069091,0.160606,0.008811,0.028314,...,0.001280,0.001508,0.000780,0.054767,0.000249,0.000058,0.001447,0.000130,0.000198,0.000069
4,0.061776,0.030888,0.048906,0.083655,0.046332,0.061776,0.045045,0.175000,0.112903,0.009753,...,0.001018,0.000447,0.000172,0.000138,0.001732,0.001241,0.000484,0.027652,0.000965,0.000112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
415313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
415314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
415315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.iloc[0]

user_id                   0.000000
region_name_0             0.000000
region_name_1             0.000000
region_name_2             0.000000
region_name_3             0.000000
                            ...   
forum.mfd.ru_frac         0.016760
yomed.ru_frac             0.001179
7info.ru_frac             0.000386
other_female_urls_frac    0.005181
other_male_urls_frac      0.000000
Name: 0, Length: 3036, dtype: float64

In [7]:
df = dd.read_parquet('..\\full_data\\figma_plan\\003cpe_manufacturer.parquet').compute()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Data columns (total 6 columns):
 #   Column                   Non-Null Count   Dtype   
---  ------                   --------------   -----   
 0   cpe_manufacturer_name_0  415317 non-null  category
 1   cpe_manufacturer_name_1  415317 non-null  category
 2   cpe_manufacturer_name_2  415317 non-null  category
 3   cpe_manufacturer_name_3  415317 non-null  category
 4   cpe_manufacturer_name_4  415317 non-null  category
 5   cpe_manufacturer_name_5  415317 non-null  category
dtypes: category(6)
memory usage: 2.4 MB
